In the last notebook, I showed that turning on the SGS and damping improves the surface winds, but the tropospheric winds are still doing crazy things. I did check that the winds in NGAqua almost obey geostrophic balance in [notebook 1.7](https://atmos.washington.edu/~nbren12/reports/uwnet/1.8-lower-atmosphere-nn.html#Prognose-U-and-V), but I will do so again here.

In [ ]:
import holoviews as hv
hv.extension('matplotlib')

%opts QuadMesh[colorbar=True](cmap='RdBu_r') 
%opts Image[colorbar=True](cmap='RdBu_r') 

In [ ]:
from uwnet.thermo import coriolis_ngaqua, ngaqua_y_to_lat, get_geostrophic_winds
from gnl.xarray import centderiv

In [ ]:
path = "/Users/noah/workspace/research/uwnet/data/runs/dosfc_dodamp_dosgs/OUT_3D/*.nc"
c_sam_2d_path = '/Users/noah/workspace/research/uwnet/data/runs/dosfc_dodamp_dosgs/OUT_2D/*.nc'
ngaqua_path = "/Users/noah/Data/2018-05-30-NG_5120x2560x34_4km_10s_QOBS_EQX/"

In [ ]:
# need to open original ng-aqua data to get PP
ds_cent = xr.open_dataset(ngaqua_path + "/coarse/3d/all.nc", chunks={'time' : 1})
ds_stag = xr.open_dataset(ngaqua_path + "/stagger/3d/all.nc", chunks={'time' : 1})

rho = xr.open_dataset(ngaqua_path + "stat.nc").RHO[0]

v_int = ds_stag.V.rename({'xc': 'x', 'ys': 'y'})

# get coriolis force
ug, vg = get_geostrophic_winds(ds_cent.PP, rho, min_cor=2e-5)

In [ ]:
hv.Dataset(vg[0].load()).to(hv.QuadMesh, ["y", "z"], dynamic=True, label="VG").redim.range(VG=(-40,40)) \
+hv.Dataset(v_int[0]).to(hv.QuadMesh, ["y", "z"], dynamic=True, label="V").redim.range(V=(-40,40))

In [ ]:
hv.Dataset(vg[0].load()).to(hv.Curve, "y", dynamic=True, label="VG").redim.range(VG=(-40,40)) *\
hv.Dataset(v_int[0].load()).to(hv.Curve, "y", dynamic=True, label="V").redim.range(VG=(-40,40)) 

In [ ]:
hv.Dataset(vg[0].mean('x')).to(hv.QuadMesh, ["y", "z"], dynamic=True, label="VG").redim.range(VG=(-10,10)) \
+hv.Dataset(v_int[0].mean('x')).to(hv.QuadMesh, ["y", "z"], dynamic=True, label="V").redim.range(V=(-10,10))

The geostrophic winds in the 

In [ ]:
coarse_sam_nn = xr.open_mfdataset(path)

ug, vg = get_geostrophic_winds(coarse_sam_nn.PP, rho)

In [ ]:
%%output size=200

hv.Dataset(vg).to.image(["x", "y"], dynamic=True).redim.range(VG=(-20,20))
# hv.Dataset(coarse_sam_nn.V).to.image(["x", "y"], dynamic=True)

In [ ]:
%%output size=200
dmap = hv.Dataset(coarse_sam_nn.V).to.image(["x", "y"], dynamic=True, label="SAM").redim.range(V=(-20,20)) +\
hv.Dataset(v_int).to.image(["x", "y"], dynamic=True, label="NGAqua").redim.range(V=(-20,20))

In [ ]:
dmap.redim.values(time=coarse_sam_nn.time, z=[11116]).cols(1)